## Leitura do arquivo

In [15]:
with open('dados/artigos.txt', 'r') as f:
  artigos = f.read()

##Tokenização

In [16]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
def separa_palavras(lista_tokens):
  lista_palavras = []
  for token in lista_tokens:
    if token.isalpha():
      lista_palavras.append(token)
  return lista_palavras

In [18]:
lista_tokens = nltk.tokenize.word_tokenize(artigos)
lista_palavras = separa_palavras(lista_tokens)
print('O número de palavras no corpus é {}.'.format(len(lista_palavras)))

O número de palavras no corpus é 403031.


In [19]:
def normalizacao(lista_palavras):
  lista_normalizada = []
  for palavra in lista_palavras:
    lista_normalizada.append(palavra.lower())
  return lista_normalizada

In [20]:
lista_normalizada = normalizacao(lista_palavras)
print('O número de palavras que o modelo vai aprender: {}.'.format(len(set(lista_normalizada))))

O número de palavras que o modelo vai aprender: 18464.


# Inserção de letras na palavra

In [21]:
def gerador_palavras(palavra):
  fatias = []
  for i in range(len(palavra)+1):
    fatias.append((palavra[:i], palavra[i:]))
  palavras_geradas = insere_letras(fatias)
  return palavras_geradas

def insere_letras(fatias):
  novas_palavras = []
  letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
  for e, d in fatias:
    for letra in letras:
      novas_palavras.append(e + letra + d)
  return novas_palavras

def corretor(palavra):
  palavras_geradas = gerador_palavras(palavra)
  palavra_correta = max(palavras_geradas, key=probabilidade)
  return palavra_correta

def probabilidade(palavras_geradas):
  frequencia = nltk.FreqDist(lista_normalizada)
  return frequencia[palavras_geradas]/total_palavras

# Teste

In [22]:
total_palavras = len(lista_normalizada)

In [23]:
palavra_exemplo = 'lgica'
corretor(palavra_exemplo)

'lógica'

# Avaliação

In [24]:
def cria_dados_teste(nome_arquivo):
  lista_palavras_teste = []
  f = open(nome_arquivo, 'r')
  for linha in f:
    correta, errada = linha.split()
    lista_palavras_teste.append((correta, errada))
  f.close()
  return lista_palavras_teste

def avaliador(testes):
  numero_palavras = len(testes)
  acertou = 0
  for correta, errada in testes:
    palavra_corrigida = corretor(errada)
    if palavra_corrigida == correta:
      acertou += 1
  taxa_acerto = round(acertou/numero_palavras * 100, 2)
  print('Taxa de acerto: {}'.format(taxa_acerto))

In [ ]:
lista_teste = cria_dados_teste('dados/palavras.txt')
avaliador(lista_teste)